In [7]:
import numpy as np
from gplearn.functions import _Function


def build_callable_program(program):
    # Check for single-node programs
    node = program[0]
    if isinstance(node, float):
        return lambda X: np.repeat(node, X.shape[0])
    if isinstance(node, int):
        return lambda X: X[:, node]

    apply_stack = []

    for node in program:
        if isinstance(node, _Function):
            apply_stack.append([node])
        else:
            # Lazily evaluate later
            apply_stack[-1].append(node)
        
        while len(apply_stack[-1]) == apply_stack[-1][0].arity + 1:
            # Apply functions that have sufficient arguments
            function = apply_stack[-1][0]
            terminals = apply_stack[-1][1:]
            intermediate_function = lambda X: function(
                *[placeholder(terminal, node)(X) for terminal in terminals])
            if len(apply_stack) != 1:
                apply_stack.pop()
                apply_stack[-1].append(intermediate_function)
            else:
                return intermediate_function
            
def placeholder(terminal, node):
    if isinstance(terminal, int):
        return lambda X: X[:, terminal]
    if isinstance(terminal, float):
        return lambda X: np.repeat(terminal, X.shape[0])
    if callable(terminal):
        print("returning callable", node, terminal)
        return terminal
    

In [8]:
from gplearn.functions import add2, mul2, neg1

program = [add2, mul2, 0.5, 1, neg1, 2] # (0.5 * x1) + (-x2)

func = build_callable_program(program)

x = np.array([[1, 2], [3, 4]])
func(x)

returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>
returning callable 2 <function build_callable_program.<locals>.<lambda> at 0x7fefa0aed6c0>

RecursionError: maximum recursion depth exceeded

In [21]:
plus = lambda x, y: x + y
neg = lambda x: -x
mult = lambda x, y: x * y

func = lambda x: plus(mult(0.5, x[:, 0]), neg(x[:, 1]))

x = np.array([[1, 2], [3, 4]])
func(x)

array([-1.5, -2.5])